In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [4]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    gram = tx.T.dot(tx)
    w_opt = np.linalg.inv(gram) * tx.T.dot(y)
    return w_opt

In [178]:
error_value=-999
def most_common(x):
    temp=np.unique(x[x!=error_value], return_counts=True)
    return temp[0][temp[1].argmax()]

def outlier_cleanup(x):
    com_vector=np.apply_along_axis(most_common, 0, x)
    com_matrix=np.repeat(com_vector[np.newaxis,:],np.array([x.shape[0]]),axis=0)
    return np.where(x!=error_value,x,com_matrix)

def pipeline(y, x):
    
    #important indices obtained through data exploration
    jet_ind=22
    global_useless=np.array([15,18,20])
    jet0_useless=jet_ind
    jet1_useless=np.array([25,jet_ind])
    jet23_useless=np.array([25,28,jet_ind])
    
    #removing all useless parameters
    x_clean=np.delete(x,global_useless,1)
    
    #splitting the samples in function of the value of PRI_jet_num 
    #then removing now useless parameters for each split
    x_jet0=np.delete(x_clean[x_clean[:,jet_ind]==0],jet0_useless-len(global_useless),1)
    x_jet1=np.delete(x_clean[x_clean[:,jet_ind]==1],jet1_useless-len(global_useless),1)
    x_jet23=np.delete(x_clean[x_clean[:,jet_ind]>1],jet23_useless-len(global_useless),1)
    
    #replacing the outliers by the most common value of their column
    x_jet0=outlier_cleanup(x_jet0)
    x_jet1=outlier_cleanup(x_jet1)
    x_jet23=outlier_cleanup(x_jet23)
    
    #splitting the y values
    y_jet0=y[x_clean[:,jet_ind]==0]
    y_jet1=y[x_clean[:,jet_ind]==1]
    y_jet23=y[x_clean[:,jet_ind]>1]
    
    return np.array([[y_jet0,x_jet0],[y_jet1,x_jet1],[y_jet23,x_jet23]])

## Generate predictions and save ouput in csv format for submission:

In [5]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [6]:
OUTPUT_PATH = '../predictions.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

NameError: name 'weights' is not defined